In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [0]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [0]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

### Задание 1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.
Для этого можно 0 заменить на крайне малое значение х, а 1 заменить на 1-х.

In [0]:
def calc_logloss(y, y_pred):
    for i in range(len(y_pred)):
        if y_pred[i] == 0:
            y_pred[i] = 1e-10
        elif y_pred[i] == 1:
            y_pred[i] = 1 - 1e-10
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [6]:
# Пример применения
y1 = np.array([1, 0], dtype = np.float64)
y_pred1 = np.array([1, 0], dtype = np.float64)
calc_logloss(y1, y_pred1)

1.000000082790371e-10

### Задание 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.
Для нахождения минимального значения log loss я добавил L2 регуляризацию для отслеживания переобучения. При, примерно, 38400 итерациях достигается минимум log loss, далее увеличивается (оставил большее значение для наглядности) 

In [0]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [8]:
def eval_model(X, y, lambda_, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * W)
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W
W = eval_model(X_st, y, lambda_=0.1, iterations=39000, alpha=1e-4)

3900 [ 0.35230568 -0.50896773  0.61707646  1.28487543] 0.679926706797577
7800 [ 0.26681776 -0.61211694  0.61886     1.16711883] 0.5994949611355047
11700 [ 0.20824826 -0.61590202  0.63035531  1.10131966] 0.5865273745929522
15600 [ 0.16137139 -0.59727746  0.64045581  1.05380279] 0.580361731162199
19500 [ 0.1214467  -0.57464215  0.64728136  1.01503588] 0.5764301202094081
23400 [ 0.08663724 -0.55293752  0.65080671  0.98184184] 0.5739171423661031
27300 [ 0.05601548 -0.53327422  0.65146475  0.95295418] 0.5724096356134503
31200 [ 0.02896075 -0.51569556  0.64976035  0.92768926] 0.5716132244399516
35100 [ 0.00497994 -0.49999398  0.64615973  0.90555497] 0.5713124462047323
39000 [-0.01634475 -0.48593318  0.64106562  0.88614261] 0.5713510780945045


### Задание 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [0]:
def calc_pred_proba(W, X):
    z = np.dot(W, X)
    y_pred_proba = sigmoid(z)
    return y_pred_proba

In [10]:
calc_pred_proba(W, X_st)

array([0.43927213, 0.50507397, 0.61930734, 0.47205166, 0.7162879 ,
       0.65439991, 0.71228378, 0.29689425, 0.42306107, 0.69362588])

### Задание 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [0]:
def calc_pred(W, X):
    n = X.shape[1]
    
    y_pred = np.zeros(n)
    z = np.dot(W, X)
    y_pred_proba = sigmoid(z)
    for i in range(y_pred_proba.shape[0]):
        if y_pred_proba[i] >= 0.5:
            y_pred[i] = 1
        elif y_pred_proba[i] < 0.5:
            y_pred[i] = 0
    return y_pred
    

In [12]:
y_pred = calc_pred(W, X_st)
y_pred

array([0., 1., 1., 0., 1., 1., 1., 0., 0., 1.])

### Задание 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [0]:
# Accuracy
def accuracy(y_, y_pred_):
    counter = 0
    for i in range(len(y_)):
        if y_[i] == y_pred_[i]:
            counter += 1
    acc = counter / len(y_)
    return acc

In [14]:
accuracy(y, y_pred)

0.7

In [0]:
# Матрица ошибок
def error_matrix(y_, y_pred_):
    TP = FP = FN = TN = 0
    for i in range(len(y)):
        if y_pred_[i] == 1 and y_[i] == 1:
            TP += 1
        elif y_pred_[i] == 1 and y_[i] == 0:
            FP += 1
        elif y_pred_[i] == 0 and y_[i] == 1:
            FN += 1
        elif y_pred_[i] == 0 and y_[i] == 0:
            TN += 1
    return TP, FP, FN, TN

In [16]:
TP, FP, FN, TN = error_matrix(y, y_pred)
print(TP, FP)
print(FN, TN)

4 2
1 3


In [0]:
# Точность
def precision(TP, FP):
    return TP / (TP + FP)

In [18]:
prec = precision(TP, FP)
prec

0.6666666666666666

In [0]:
# Полнота
def recall(TP, FN):
    return TP / (TP + FN)

In [20]:
rec = recall(TP, FN)
rec

0.8

In [0]:
# F1 score
def F1(prec_, rec_):
    return 2 * prec_ * rec_ / (prec_ + rec_)

In [22]:
F1(prec, rec)

0.7272727272727272

### Задание 6. Могла ли модель переобучиться? Почему?
Да, может. Если не делать регуляризацию, то видно как уменьшается ошибка при увеличении количества итераций, но при этом веса признаков становятся огромными.

### Задание 7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [23]:
def eval_model_l1(X, y, lambda_, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_/2 * np.sign(W))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W
W = eval_model_l1(X_st, y, lambda_=0.1, iterations=39000, alpha=1e-4)

3900 [ 0.34766973 -0.50868291  0.62167531  1.31689266] 0.689062978951687
7800 [ 0.25156984 -0.6259443   0.62687299  1.22104767] 0.5998794781737434
11700 [ 0.18005463 -0.64048724  0.64301706  1.17484529] 0.5838095477229925
15600 [ 0.11853861 -0.62990544  0.65840861  1.14535801] 0.5752282405854603
19500 [ 0.06258281 -0.6132923   0.67075469  1.12325721] 0.5689510763808466
23400 [ 0.01057162 -0.59621473  0.67969824  1.10547582] 0.5641393751558492
27300 [-0.00832512 -0.58191946  0.68562502  1.08958682] 0.5626137357781036
31200 [-0.01820638 -0.57157868  0.68961691  1.07367958] 0.5620162436022527
35100 [-0.02704015 -0.56281868  0.69222863  1.05864386] 0.5616386642311361
39000 [-0.03483239 -0.55490414  0.69372095  1.04465476] 0.5614450167587477


In [24]:
def eval_model_l2(X, y, lambda_, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * W)
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W
W = eval_model_l2(X_st, y, lambda_=0.1, iterations=39000, alpha=1e-4)

3900 [ 0.35230568 -0.50896773  0.61707646  1.28487543] 0.679926706797577
7800 [ 0.26681776 -0.61211694  0.61886     1.16711883] 0.5994949611355047
11700 [ 0.20824826 -0.61590202  0.63035531  1.10131966] 0.5865273745929522
15600 [ 0.16137139 -0.59727746  0.64045581  1.05380279] 0.580361731162199
19500 [ 0.1214467  -0.57464215  0.64728136  1.01503588] 0.5764301202094081
23400 [ 0.08663724 -0.55293752  0.65080671  0.98184184] 0.5739171423661031
27300 [ 0.05601548 -0.53327422  0.65146475  0.95295418] 0.5724096356134503
31200 [ 0.02896075 -0.51569556  0.64976035  0.92768926] 0.5716132244399516
35100 [ 0.00497994 -0.49999398  0.64615973  0.90555497] 0.5713124462047323
39000 [-0.01634475 -0.48593318  0.64106562  0.88614261] 0.5713510780945045
